
# Preparacion y limpieza de datos

Comenzamos importando las librerías de carga de datos y de visualización.

In [117]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
import os
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

Luego definiremos los tipos de las variables y ademas se cargaran los datos


In [118]:
# Leer los datos y establecer los tipos de datos para cada columna basado en la información del pdf
dtypes = {'Vlo-I': str, 'Ori-I': str, 'Des-I': str, 'Emp-I': str,
          'Vlo-O': str, 'Ori-O': str, 'Des-O': str, 'Emp-O': str,
          'DIA': int, 'MES': int, 'AÑO': int, 'DIANOM': str, 'TIPOVUELO': str,
          'OPERA': str, 'SIGLAORI': str, 'SIGLADES': str, 'Temporada alta': int,
          'Diferencia en minutos': int, 'Atraso menor': int, 'Periodo día': str}

df = pd.read_csv('../data/synthetic_features.csv', dtype=dtypes, parse_dates=['Fecha-I', 'Fecha-O'])


Comenzamos eliminando la informacion repetida segun la etapa de exploracion de datos


In [119]:
df.drop(['Vlo-O', 'Des-O', 'SIGLAORI', 'Ori-O','Ori-I'], axis=1, inplace=True)

Ahora eliminaremos las columnas que no disponemos al ahora de la prediccion, que serian las relacionadas a la operacion.

In [120]:
df.drop(['Fecha-O', 'Emp-O', 'DIA', 'MES','AÑO',], axis=1, inplace=True)

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68205 entries, 0 to 68204
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Fecha-I                68205 non-null  datetime64[ns]
 1   Vlo-I                  68205 non-null  object        
 2   Des-I                  68205 non-null  object        
 3   Emp-I                  68205 non-null  object        
 4   DIANOM                 68205 non-null  object        
 5   TIPOVUELO              68205 non-null  object        
 6   OPERA                  68205 non-null  object        
 7   SIGLADES               68205 non-null  object        
 8   Temporada alta         68205 non-null  int64         
 9   Diferencia en minutos  68205 non-null  int64         
 10  Atraso menor           68205 non-null  int64         
 11  Periodo día            68205 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(8)
memory usage: 6.2+

In [122]:
df.nunique()

Fecha-I                  53252
Vlo-I                      584
Des-I                       64
Emp-I                       30
DIANOM                       7
TIPOVUELO                    2
OPERA                       23
SIGLADES                    62
Temporada alta               2
Diferencia en minutos      176
Atraso menor                 2
Periodo día                  3
dtype: int64

Como se menciono en la etapa de exploracion de datos, un atraso menor sera considerado como un atraso de todas maneras.

In [123]:
df['Label'] = (df['Diferencia en minutos'] > 0).astype(int)

Ahora que tenemos una etiqueta para nuestros datos, ya no necesitaremos la variable diferencia en minutos y el tipo de atraso, ya que esta variable es informacion que no tendremos a la hora de crear predicciones en el mundo real.

In [124]:
df.drop(['Diferencia en minutos','Atraso menor'], axis=1, inplace=True)

In [125]:
df.nunique()

Fecha-I           53252
Vlo-I               584
Des-I                64
Emp-I                30
DIANOM                7
TIPOVUELO             2
OPERA                23
SIGLADES             62
Temporada alta        2
Periodo día           3
Label                 2
dtype: int64

# Codificacion de variables categoricas

Ahora que tenemos solo los datos de los cuales disponemos a la hora de crear predicciones, podemos comenzar a codificar las variables categoricas. Para esto primero debemos ver que variables son categoricas y si son ordinales o no. <br>

- **Categoricas Ordinales**: Son aquellas que tienen un orden logico, por ejemplo, el tipo de atraso, ya que un atraso de 5 minutos es menor que un atraso de 10 minutos. <br>

- **Categoricas No Ordinales**: Son aquellas que no tienen un orden logico, por ejemplo, el dia de la semana, ya que no hay un dia de la semana que sea mayor que otro. <br>

Con esto en mente comenzamos a clasificar las variables categoricas.

- __Vlo-I__: Categorica no ordinal, dado que los codigos de vuelono representan un orden logico. <br>
- __Des-I__: Categorica no ordinal, dado que los codigos de  ciudad no representan un orden logico. <br>
- __Emp-I__: Categorica no ordinal, dado que los codigos de aerolinea no representan un orden logico. <br>
- __TIPOVUELO__: Categorica no ordinal, ya que los valores "I" y "N" representan tipos de vuelo distintos sin un orden lógico entre ellos. <br>
- __OPERA__: Categorica no ordinal, dado que los nombres de aerolínea no representan un orden lógico.
- __Temporada alta__: Categorica no ordinal, dado que es solo un indicador del tipo de temporada. <br>
- __SIGLADES__: Categorica no ordinal, dado que el nombrede  la  ciudad no representan un orden logico. <br>

Las siguientes variables son categorica, pero presentan el problema de una caracteristica ciclica/ordinal. Para analizarlas se puede utilizar tanto una codificacion ordinal como una codificacion ciclica, en este caso se __asumiran ordinales__ . <br>

- __Periodo día__: Categorica ordinal. <br>
- __DIANOM__: Categorica no ordinal, ya que es una representacion en palabras de un dia. <br>


Finalmente, la fecha (Fecha-I), es una variable ciclica, por lo que para representarla como tal se descompondra en mes, dia, hora y minutos. Para luego codificarla como una variable ciclica descomponiendo cada una de estas variables en senos y cosenos. [1] <br>

[1]: https://developer.nvidia.com/blog/three-approaches-to-encoding-time-information-as-features-for-ml-models/



Comenzamos por aplicar one-hot encoding a las variables categoricas no ordinales y guardamoslos resultados en un nuevo dataframe para evitar confusiones.

In [126]:
# Seleccionar las columnas a codificar
cols_to_encode = ['Vlo-I', 'Des-I', 'Emp-I', 'TIPOVUELO', 'OPERA', 'Temporada alta', 'SIGLADES']

# Crear un objeto OneHotEncoder
encoder = OneHotEncoder()

# Ajustar el codificador a los datos de entrenamiento
encoder.fit(df[cols_to_encode])

# Codificar las columnas seleccionadas
encoded_cols = encoder.transform(df[cols_to_encode]).toarray()

# Crear un DataFrame con las columnas codificadas
encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(cols_to_encode))

# Convertir los valores booleanos a enteros
encoded_df = encoded_df.astype(int)

# Concatenar las columnas codificadas con el DataFrame original
new_df = pd.concat([df, encoded_df], axis=1)

# Eliminar las columnas originales que fueron codificadas
new_df.drop(cols_to_encode, axis=1, inplace=True)

# Guardar el codificador en un archivo
if not os.path.exists('../encoders'):
    os.makedirs('../encoders')
with open('../encoders/one-hot-encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)


Ahora aplicamos label encoding a las variables categoricas ordinales.

In [127]:
# Seleccionar las columnas a codificar
cols_to_encode = ['Periodo día', 'DIANOM']

# Definir las categorías ordinales y su orden
periodo_dia_categories = ['mañana', 'tarde', 'noche']
dianom_categories = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']

# Crear un objeto OrdinalEncoder
encoder = OrdinalEncoder(categories=[periodo_dia_categories, dianom_categories], dtype=int)

# Ajustar el codificador a los datos de entrenamiento
encoder.fit(new_df[cols_to_encode])

# Codificar las columnas seleccionadas
encoded_cols = encoder.transform(new_df[cols_to_encode])

# Crear un DataFrame con las columnas codificadas
encoded_df = pd.DataFrame(encoded_cols, columns=['Periodo dia_encoded', 'DIANOM_encoded'])

# Concatenar las columnas codificadas con el DataFrame original
new_df = pd.concat([new_df, encoded_df], axis=1)

# Eliminar las columnas originales que fueron codificadas
new_df.drop(cols_to_encode, axis=1, inplace=True)

# Guardar el codificador en un archivo
if not os.path.exists('../encoders'):
    os.makedirs('../encoders')
with open('../encoders/ordinal-encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)

Ahora trabajaremos la Fecha-I, para esto primero la descomponemos en mes, dia, hora y minutos. (Estamos ignorando el año ya que todos los datos son del mismo año, a excepcion de dos muestras)

In [128]:
# Convertir la columna Fecha-I a tipo de datos datetime
new_df['Fecha-I'] = pd.to_datetime(new_df['Fecha-I'])

# Extraer el mes, día, hora y minuto de la columna Fecha-I
new_df['Mes'] = new_df['Fecha-I'].dt.month
new_df['Día'] = new_df['Fecha-I'].dt.day
new_df['Hora'] = new_df['Fecha-I'].dt.hour
new_df['Minuto'] = new_df['Fecha-I'].dt.minute

# Eliminar la columna original Fecha-I
new_df.drop('Fecha-I', axis=1, inplace=True)


Ahora que tenemos las variables descompuestas, las codificamos como variables ciclicas.

In [129]:
def encode(data, col, max_val):
    """Codifica la columna dada utilizando las funciones seno y coseno.

    Args:
        data: El DataFrame que se va a codificar.
        col: El nombre de la columna que se va a codificar.
        max_val: El valor máximo de la columna.

    Returns:
        El DataFrame con la columna codificada.
    """

    # Primero, calculamos el seno y coseno de los valores de la columna.

    data[col + '_sin'] = np.sin(2 * np.pi * data[col] / max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col] / max_val)

    # Finalmente, retornamos el DataFrame con la columna codificada.

    return data

In [130]:
# Codificar las columnas Month, Day, Hour y Minute
new_df = encode(new_df, 'Mes', 12)
new_df = encode(new_df, 'Día', 31)
new_df = encode(new_df, 'Hora', 24)
new_df = encode(new_df, 'Minuto', 60)

# Eliminar las columnas originales Mes, Día, Hora y Minuto
new_df.drop(['Mes', 'Día', 'Hora', 'Minuto'], axis=1, inplace=True)

# Set de entrenamineto y prueba

Ahora que todas las variables han sido codificadas, separaremos los datos en un set de entrenamiento y un set de prueba, ya que sera importante aislar el set de prueba de las transformaciones que se haran a continuacion.

In [131]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(new_df.drop('Label', axis=1), new_df['Label'], test_size=0.2, random_state=42)

# Imprimir las dimensiones de los conjuntos de entrenamiento y prueba
print('Forma de X_train:', X_train.shape)
print('Forma de y_train:', y_train.shape)
print('Forma de X_test:', X_test.shape)
print('Forma de y_test:', y_test.shape)


Forma de X_train: (54564, 777)
Forma de y_train: (54564,)
Forma de X_test: (13641, 777)
Forma de y_test: (13641,)


Ahora que tenemos los datos separados, abordaremos el desbalance de clases, ya que hay mas vuelos atrasados que vuelos no atrasados.

In [132]:
# Verificar el equilibrio de las etiquetas en el conjunto de entrenamiento
train_counts = y_train.value_counts()
train_pct = train_counts / len(y_train) * 100
print('Recuento de etiquetas en el conjunto de entrenamiento:')
print(train_counts)
print('Porcentajes de etiquetas en el conjunto de entrenamiento:')
print(train_pct)

# Verificar el equilibrio de las etiquetas en el conjunto de prueba
test_counts = y_test.value_counts()
test_pct = test_counts / len(y_test) * 100
print('Recuento de etiquetas en el conjunto de prueba:')
print(test_counts)
print('Porcentajes de etiquetas en el conjunto de prueba:')
print(test_pct)


Recuento de etiquetas en el conjunto de entrenamiento:
Label
1    36336
0    18228
Name: count, dtype: int64
Porcentajes de etiquetas en el conjunto de entrenamiento:
Label
1    66.593358
0    33.406642
Name: count, dtype: float64
Recuento de etiquetas en el conjunto de prueba:
Label
1    9170
0    4471
Name: count, dtype: int64
Porcentajes de etiquetas en el conjunto de prueba:
Label
1    67.223811
0    32.776189
Name: count, dtype: float64


Antes de continuar guardaremos los datos para poderhacer pruebas con y sin el desbalance de clases.

In [133]:
# Save the training set to a CSV file
X_train.to_csv('../data/X_train_desbalanceado.csv', index=False)
y_train.to_csv('../data/y_train_desbalanceado.csv', index=False)

# Save the test set to a CSV file
X_test.to_csv('../data/X_test_desbalanceado.csv', index=False)
y_test.to_csv('../data/y_test_desbalanceado.csv', index=False)

Ahora que confirmamos el debalance de clases en ambos sets de datos, vamos a tomar los vuelos atrasados en entrenamiento y los moveremos al set de datos de prueba, demanera que el set de entrenamiento sea balanceado.

In [134]:
# Encontrar la etiqueta de la clase mayoritaria
majority_label = train_counts.idxmax()

# Encontrar el número de muestras a mover al conjunto de prueba
num_to_move = train_counts[majority_label] - train_counts.min()

# Seleccionar las muestras excedentes con la clase mayoritaria en el conjunto de entrenamiento
excess_samples = X_train[y_train == majority_label].iloc[:num_to_move]

# Mover las muestras excedentes al conjunto de prueba
X_test = pd.concat([X_test, excess_samples])
y_test = pd.concat([y_test, y_train[y_train == majority_label].iloc[:num_to_move]])

# Eliminar las muestras excedentes del conjunto de entrenamiento
X_train = X_train.drop(excess_samples.index)
y_train = y_train.drop(excess_samples.index)


In [135]:
# Verificar el equilibrio de las etiquetas en el conjunto de entrenamiento
train_counts = y_train.value_counts()
train_pct = train_counts / len(y_train) * 100
print('Recuento de etiquetas en el conjunto de entrenamiento:')
print(train_counts)
print('Porcentajes de etiquetas en el conjunto de entrenamiento:')
print(train_pct)

# Verificar el equilibrio de las etiquetas en el conjunto de prueba
test_counts = y_test.value_counts()
test_pct = test_counts / len(y_test) * 100
print('Recuento de etiquetas en el conjunto de prueba:')
print(test_counts)
print('Porcentajes de etiquetas en el conjunto de prueba:')
print(test_pct)


Recuento de etiquetas en el conjunto de entrenamiento:
Label
0    18228
1    18228
Name: count, dtype: int64
Porcentajes de etiquetas en el conjunto de entrenamiento:
Label
0    50.0
1    50.0
Name: count, dtype: float64
Recuento de etiquetas en el conjunto de prueba:
Label
1    27278
0     4471
Name: count, dtype: int64
Porcentajes de etiquetas en el conjunto de prueba:
Label
1    85.917667
0    14.082333
Name: count, dtype: float64


In [136]:
# Save the training set to a CSV file
X_train.to_csv('../data/X_train_balanceado.csv', index=False)
y_train.to_csv('../data/y_train_balanceado.csv', index=False)

# Save the test set to a CSV file
X_test.to_csv('../data/X_test_balanceado.csv', index=False)
y_test.to_csv('../data/y_test_balanceado.csv', index=False)